In [1]:
import numpy as np
import datajoint as dj

In [2]:
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

Connecting celiib@10.28.0.34:3306


In [3]:
#try to get the basketball cells by finding those whose automatic segmentation has more labels greater than 16
segment_ids,triangles_total = ta3p100.Annotation.fetch("segment_id","triangles")

In [4]:
True in triangles_total[0] > 16

False

In [5]:
from collections import Counter
from tqdm import tqdm

size_threshold = 50

possible_basketball = []

for id,tri in tqdm(zip(segment_ids,triangles_total)):
    
    if len(tri) > size_threshold and (True in (tri > 16)):
        possible_basketball.append(id)

print(possible_basketball)

322it [00:00, 600.74it/s]

[648518346341388820, 648518346349386137, 648518346349471156, 648518346349471500, 648518346349471910, 648518346349472574, 648518346349473044, 648518346349473160, 648518346349473583, 648518346349473597, 648518346349473781, 648518346349473804, 648518346349473815, 648518346349473816, 648518346349473819, 648518346349473822, 648518346349473832, 648518346349473833, 648518346349473835, 648518346349473847, 648518346349473848, 648518346349473907, 648518346349474146, 648518346349475035, 648518346349475118, 648518346349475120, 648518346349475132, 648518346349475138, 648518346349475150, 648518346349475425, 648518346349475436, 648518346349475518, 648518346349475522, 648518346349475523, 648518346349475524, 648518346349475525, 648518346349475526, 648518346349475530, 648518346349475531, 648518346349475534, 648518346349475536, 648518346349475540, 648518346349475573, 648518346349476159, 648518346349476961, 648518346349477981, 648518346349478197, 648518346349478330, 648518346349478380, 648518346349478473,

In [6]:
print(len(possible_basketball))

165


In [7]:
possible_basketball[6]

648518346349473044

In [8]:
"""
List of the neeurons and their category

"""

no_basketball_with_soma = [648518346341388820,
                             ]

no_basketball_no_soma = [
    
]

basketball_cell = [648518346349471156,
                   648518346349471500,
                   648518346349471910,
                   648518346349472574,
                   648518346349473044
    
]

small_chunk_with_soma = [
    648518346349386137,
]

In [27]:
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,vertices=[], triangles=[]):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    
    file_loc = pathlib.Path.cwd() / "temp"
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

# Try and see what Pymesh Fix does

In [51]:
segment_id=648518346349471500

In [52]:
basketball_mesh = (ta3p100.CleansedMesh & dict(segment_id=segment_id)).fetch1()
verts = basketball_mesh["vertices"]
faces = basketball_mesh["triangles"]

In [53]:
write_Whole_Neuron_Off_file(str(segment_id)+ "_original",verts,faces)

Done writing OFF file


('/notebooks/19_Whole_Neuron_Seg_Revisited/temp/neuron_648518346349471500_original',
 'neuron_648518346349471500_original',
 '/notebooks/19_Whole_Neuron_Seg_Revisited/temp')

In [54]:
import pymeshfix
import time

#print("Step 2: Remove all error semgents")
start_time = time.time()
#pass the vertices and faces to pymeshfix to become watertight
meshfix = pymeshfix.MeshFix(verts,faces)
meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)
print(f"Step 2: Pymesh shrinkwrapping: {time.time() - start_time}")


Step 2: Pymesh shrinkwrapping: 186.20285940170288


In [55]:
#print("Step 2: Writing Off File")
start_time = time.time()
#write the new mesh to off file
path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(segment_id),meshfix.v,meshfix.f)
print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")


Done writing OFF file
Step 3: Writing shrinkwrap off file: 2.1492819786071777


In [ ]:
import os
import contextlib

with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    part_2_spines_whole.export("./neurons/part_2_spines_whole.off")
    part_3_spines_whole.export("./neurons/part_3_spines_whole.off")
    part_4_spines_whole.export("./neurons/part_4_spines_whole.off")
    part_1_spines_whole.export("./neurons/part_1_spines_whole.off")